<a href="https://colab.research.google.com/github/cookie-pan/Python-Data-Visualization/blob/main/project/project_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

"""
Project for Week 3 of "Python Data Visualization".
Unify data via common country name.
Be sure to read the project description page for further information
about the expected behavior of the program.
author: Pan_xin
"""

import csv
import math
import pygal
 


def reconcile_countries_by_name(plot_countries, gdp_countries):
    """
    Inputs:
      plot_countries - Dictionary whose keys are plot library country codes
                       and values are the corresponding country name
      gdp_countries  - Dictionary whose keys are country names used in GDP data
    Output:
      A tuple containing a dictionary and a set.  The dictionary maps
      country codes from plot_countries to country names from
      gdp_countries The set contains the country codes from
      plot_countries that were not found in gdp_countries.
    """
    
    country_codes = set()
    country_datas = {}
    
    for country_code in plot_countries: 
        if plot_countries.get(country_code) in gdp_countries: 
            country_datas[country_code] = plot_countries[country_code]
            
        else:
            country_codes.add(country_code)
            
    tupled_values = () + (country_datas, country_codes,) 

    return tupled_values

    
def build_map_dict_by_name(gdpinfo, plot_countries, year):
    """
    Inputs:
      gdpinfo        - A GDP information dictionary
      plot_countries - Dictionary whose keys are plot library country codes
                       and values are the corresponding country name
      year           - String year to create GDP mapping for
    Output:
      A tuple containing a dictionary and two sets.  The dictionary
      maps country codes from plot_countries to the log (base 10) of
      the GDP value for that country in the specified year.  The first
      set contains the country codes from plot_countries that were not
      found in the GDP data file.  The second set contains the country
      codes from plot_countries that were found in the GDP data file, but
      have no GDP data for the specified year.
    """
    dict_code_gdp={}
    extra_code=set()
    unex_coun_code=set()
    csv_dict={}
    
    with open (gdpinfo["gdpfile"]) as origin:
        file=csv.DictReader(origin,delimiter=gdpinfo["separator"], quotechar=gdpinfo["quote"])

        for single in file:
            single_dict=dict(single)
            csv_dict[single_dict[gdpinfo["country_name"]]]=single_dict

    for item in plot_countries:

        if plot_countries[item] in csv_dict:
            try:
                dict_code_gdp[item]=math.log10(float(csv_dict[plot_countries[item]][year]))
            
            except ValueError:
                unex_coun_code.add(item)

        else:
            extra_code.add(item) 

    returned_tuple=(dict_code_gdp,extra_code,unex_coun_code)
    return(returned_tuple)


def render_world_map(gdpinfo, plot_countries, year, map_file):
    """
    Inputs:
      gdpinfo        - A GDP information dictionary
      plot_countries - Dictionary whose keys are plot library country codes
                       and values are the corresponding country name
      year           - String year to create GDP mapping for
      map_file       - Name of output file to create
    Output:
      Returns None.
    Action:
      Creates a world map plot of the GDP data for the given year and
      writes it to a file named by map_file.
    """
    data=build_map_dict_by_name(gdpinfo, plot_countries, year)
    ex_code=data[0]
    unex_coun=data[1]
    unex_gdp=data[2]

    worldmap_chart = pygal.maps.world.World()
    worldmap_chart.title = 'GDP by country for ' + year + ' (log scale), unified by common' + \
                           ' country NAME'
    worldmap_chart.add('GDP For ' + year, ex_code)
    worldmap_chart.add('Missing from World Bank Data',list(unex_coun))
    worldmap_chart.add('No GDP Data', list(unex_gdp))

    worldmap_chart.render_to_file(map_file)